In [1]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

In [3]:
key = os.getenv("OPENAI_API_KEY")

In [4]:
print(key)

sk-proj-sNiKBrmregdyYDknOHAFT3BlbkFJ1a9RWOW70w8IuZtDoP4v


In [5]:
llm = ChatOpenAI(openai_api_key=key, temperature=0.7, model_name="gpt-3.5-turbo")

/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f7f0c2448b0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f7f0c251700>, openai_api_key='sk-proj-sNiKBrmregdyYDknOHAFT3BlbkFJ1a9RWOW70w8IuZtDoP4v', openai_proxy='')

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [11]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [13]:
quiz_generation_promt = PromptTemplate(
    input_variables= ['text', 'number', 'subject', 'tone', 'response_json'],
    template = TEMPLATE
)

In [18]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_promt, output_key='quiz', verbose=True)


In [16]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [17]:
quiz_evaluation_prompt =  PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE2
)

In [19]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review', verbose=True)

In [20]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"], 
    verbose=True)

In [21]:
filepath="/Users/dacenish/Documents/Backup/danish backup/LLM Projects/MCQ Generator/data.txt"
with open(filepath, 'r') as file:
    TEXT = file.read()

In [25]:
print(TEXT)

In [26]:
response_dump = json.dumps(RESPONSE_JSON)
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [27]:
# To track the usgae of Token
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": response_dump 
        }
    )

/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:
You are an expert MCQ maker. Given the above text, it is your job to create a quiz  of 5 multiple choice questions for biology students in simple tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make 5 MCQs
### RESPONSE_JSON
{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}



> Finished cha

In [29]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1148
Prompt Tokens:731
Completion Tokens:417
Total Cost:0.0019305


In [34]:
quiz = response.get('quiz')

In [37]:
quiz = json.loads(quiz)

In [38]:
 
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [41]:
quiz = pd.DataFrame(quiz_table_data)

In [42]:
quiz.to_csv('machinelearning.csv')